In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import matplotlib.pyplot as plt
import torch 
from torch import nn
torch.set_default_dtype(torch.float64)
import torch.nn.functional as func

import copy
from tqdm.notebook import tqdm
import time
import random
import seaborn as sns
sns.set_theme()

from sklearn.neighbors import NearestNeighbors
from scipy.interpolate import griddata
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import euclidean_distances
from IPython.display import clear_output

In [ ]:
plt.rcParams['text.usetex'] = True

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import sys; sys.path.insert(1, '../')
from committor_lib.Example3 import Solvated_dimer,Model,Solver
from committor_lib.utils    import rL2,LangevinIntegrator,Metadynamics,Metadynamics_Extend

# The system

In [ ]:
# n_atoms = 32; L = 64**(1/3); density is 0.5

SYS      = Solvated_dimer(n_atoms=32,L=64**(1/3),h=5,w=0.25,r0=2**(1/6),eps=1,rWCA=2**(1/6),); EPS = 1;
get_x_bc = SYS.get_X_bc
LI       = LangevinIntegrator(dim=SYS.dim)

# Read some data

In [ ]:
# By simulating the Langevin dynamics
X_A,X_B = np.loadtxt('../saved_data/X_A'),np.loadtxt('../saved_data/X_B')
# By Monte Carlo methods
X_u,q_u = np.loadtxt('../saved_data/X_u'),np.loadtxt('../saved_data/q_u')
print(X_A.shape,X_B.shape,X_u.shape,q_u.shape);
def Error_Model(model): return rL2(q_u,model.get_q(X_u))
def E_AB(model): return np.sqrt(np.mean(model.get_q(X_A)**2))+np.sqrt(np.mean((1-model.get_q(X_B))**2))

In [ ]:
def Estimate_Model(model,X_u=X_u,q_u=q_u,SYS=SYS):
    fig,ax = plt.subplots(1,2,figsize=(12,5))
    q_NN = model.get_q(X_u)
    ax[0].scatter(SYS.get_bond_length(X_u),q_NN,s=5.,label='100 states')
    ax[1].scatter(SYS.get_bond_length(X_u),q_u,s=5.,c='g',label='100 states')
    ax[0].set_title('NN: %.3f'%rL2(q_u,q_NN))
    ax[1].set_title('MC')
    for i in range(2):
        ax[i].set_xlabel('bond length',fontsize=13)
        ax[i].set_ylabel('$q$',fontsize=13,rotation=0)
        ax[i].set_xlim([SYS.r0,SYS.r0+2*SYS.w])
        ax[i].set_ylim([-.1,1.1])
        ax[i].legend(fontsize=13)
    plt.show()

# Set the model

In [ ]:
def get_wrapvec(x,L=SYS.L): return x-torch.round(x/L)*L

n           = 10
rbf_centers = torch.tensor(np.linspace(0,SYS.L/2*np.sqrt(3)+1,100)).cuda()
rbf_gap     = (rbf_centers[1]-rbf_centers[0])**2*10
model       = Model(get_wrapvec=get_wrapvec,rbf_centers=rbf_centers,rbf_gap=rbf_gap,
                    n_atoms=SYS.n_atoms,input_dim=SYS.dim,n_fea=32,n_interactions=2,n=n).cuda()
solver      = Solver(model,unit_len=100,q0=-5,q1=5)

In [ ]:
Estimate_Model(model,X_u=X_u,q_u=q_u,SYS=SYS)

# Train

In [ ]:
def show_distr(X,ax,get_phi=SYS.get_bond_length):
    bond_length = get_phi(X)
    ax.hist(bond_length,25)
    ax.axvline(SYS.r0,linestyle='--',linewidth=3,color='r')
    ax.axvline(SYS.r0+2*SYS.w,linestyle='--',linewidth=3,color='r')
    ax.set_xlabel('bond length')
    ax.set_title('shape:'+str(X.shape))

In [ ]:
def down_sample(X):
    mask = SYS.IsInA(X) | SYS.IsInB(X)
    return X[~mask]
def get_train_test(X,coef,X_A,X_B,ratio=0.7):
    Xc         = np.hstack([X,coef.reshape(-1,1)])
    data_train = []
    data_test  = []
    for d,requires_grad in [[Xc,True],[X_A,False],[X_B,False]]:
        perm = np.random.permutation(len(d))
        d1 = d[perm[:int(len(d)*ratio)]]
        data_train.append(torch.tensor(d1,requires_grad=requires_grad).cuda())
        d2 = d[perm[int(len(d)*ratio):]]
        data_test.append(torch.tensor(d2,requires_grad=requires_grad).cuda())
    return data_train,data_test 

In [ ]:
NAME = 'DIM2'
if not os.path.isdir('../saved_models/'+NAME+'/'): os.mkdir('../saved_models/'+NAME+'/')
if not os.path.isdir('../saved_data/'+NAME+'/'): os.mkdir('../saved_data/'+NAME+'/')

In [ ]:
def get_wrapvec(x,L=SYS.L): return x-torch.round(x/L)*L

n           = 10
rbf_centers = torch.tensor(np.linspace(0,SYS.L/2*np.sqrt(3)+1,100)).cuda()
rbf_gap     = (rbf_centers[1]-rbf_centers[0])**2*10
model       = Model(get_wrapvec=get_wrapvec,rbf_centers=rbf_centers,rbf_gap=rbf_gap,
                    n_atoms=SYS.n_atoms,input_dim=SYS.dim,n_fea=32,n_interactions=2,n=n).cuda()
solver      = Solver(model,unit_len=100,q0=-5,q1=5)

meta = Metadynamics_Extend(model=model,h=.1,w=.003,eps=EPS)

In [ ]:
#########     training details  ##########################
c1 = 1;  c2 = 1; learning_rate = 1e-4;                   #
##########################################################

model.load_state_dict(torch.load('../saved_models/E3_Random1'))
torch.save(meta,'../saved_models/'+NAME+'/meta%d'%0)
torch.save(model.state_dict(), '../saved_models/'+NAME+'/par%d'%0)
    
K = 10
for k in range(1,K+1):
    
    meta.re_init()
    meta.perform(dV=SYS.get_dV,x=X_A[-1].reshape(1,-1),dt=1e-4,eps=EPS,
                 N=int(2e5),N_add=200,show_freq=.1,use_tqdm=False,get_x_bc=get_x_bc,show_distr=show_distr,
                 fig_name='../saved_data/'+NAME+'/ite%d'%k)
    meta.show_meta(show_distr)
    
    def get_f(X): return -SYS.get_dV(X) + meta.get_dF(X)/2
    X0  = np.repeat(X_A[-1].reshape(1,-1),100,axis=0)  
    X   = LI.get_data(X0,get_f,eps=EPS,dt=1e-4,m=100,T0=1,T=6,get_x_bc=get_x_bc,use_tqdm=False);
    X   = down_sample(X)
    fig,ax = plt.subplots(1,1,figsize=(3,3))
    show_distr(X,ax)
    plt.show()
    
    V_add = -meta.get_F(X)/2
    coef  = np.exp(1/EPS*(V_add-V_add.max()))
    coef  = coef/coef.mean()
    
    data_train,data_test = get_train_test(X,coef,X_A,X_B,ratio=.99)
    for i in range(len(data_train)): print(data_train[i].shape,data_test[i].shape)
    optimizer = torch.optim.Adam(model.parameters(),lr=torch.tensor(learning_rate).cuda())
    solver.train_model(data_train=data_train,data_test=data_test,c1=c1,c2=c2,batch_size=100,
                       optimizer=optimizer,n_steps=int(5000+1),n_show_loss=500,use_tqdm=False,
                       error_model1=Error_Model,error_model2=E_AB,)
    Estimate_Model(model)
    
    torch.save(meta,'../saved_models/'+NAME+'/meta%d'%k)
    torch.save([X,coef],'../saved_data/'+NAME+'/data%d'%k)
    torch.save(model.state_dict(), '../saved_models/'+NAME+'/par%d'%k)